In [ ]:
#@title 🎧 Code Walkthrough: Code Sliding Window
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_04_code_sliding_window.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction: Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_01_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


# Dataset and DataLoader -- From Tokens to Training Batches -- Vizuara

> **What you will build:** A PyTorch Dataset and DataLoader that converts raw token sequences into shuffled, batched input-target pairs ready for language model training.

In [ ]:
#@title 🎧 Listen: Why Matter
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_02_why_matter.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 1. Why Does This Matter?

The tokenizer gave us a sequence of numbers. But a language model does not train on a single long sequence -- it needs structured **input-target pairs** organized into **batches**. This is where the Dataset and DataLoader come in.

The Dataset creates training pairs using a sliding window: "given these tokens, predict the next one." The DataLoader then organizes these pairs into efficient batches, shuffles them, and streams them to the GPU.

Together, they form the conveyor belt of our training factory -- without them, the model has nothing to learn from.

In [ ]:
#@title 🎧 Listen: Intuition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_03_intuition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 2. Building Intuition

Let us start with the core idea. A language model learns by predicting the next token. Given the sequence `[The, cat, sat, on, the, mat]`, we create multiple training examples:

- **Input:** `[The, cat, sat, on]` --> **Target:** `[cat, sat, on, the]`
- **Input:** `[cat, sat, on, the]` --> **Target:** `[sat, on, the, mat]`

Notice: the target is just the input shifted by one position. Each position in the input is trained to predict the next token.

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# Simple demonstration of the sliding window
tokens = ["The", "cat", "sat", "on", "the", "mat"]
token_ids = list(range(len(tokens)))  # [0, 1, 2, 3, 4, 5]
context_length = 4

print("Sliding Window Examples:")
print("=" * 60)
for i in range(len(tokens) - context_length):
    input_tokens = tokens[i:i + context_length]
    target_tokens = tokens[i + 1:i + context_length + 1]
    print(f"  Input:  {input_tokens}")
    print(f"  Target: {target_tokens}")
    print(f"  (Each input token predicts the token directly to its right)")
    print()

print(f"Total training samples: {len(tokens) - context_length}")

In [ ]:
#@title 🎧 What to Look For: Viz Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_05_viz_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Let us visualize the sliding window mechanism.

In [ ]:
#@title 🎧 What to Look For: Viz Sliding Window
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_06_viz_sliding_window.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))

# Draw the full token sequence
colors = plt.cm.Set3(np.linspace(0, 1, len(tokens)))
for i, (tok, color) in enumerate(zip(tokens, colors)):
    rect = plt.Rectangle((i * 1.5, 4), 1.2, 0.8, facecolor=color, edgecolor='black', linewidth=1.5)
    ax.add_patch(rect)
    ax.text(i * 1.5 + 0.6, 4.4, tok, ha='center', va='center', fontsize=12, fontweight='bold')
    ax.text(i * 1.5 + 0.6, 3.5, f'[{i}]', ha='center', va='center', fontsize=10, color='gray')

ax.text(-0.5, 4.4, 'Full sequence:', ha='right', va='center', fontsize=11, fontweight='bold')

# Draw sliding windows
for win_idx in range(len(tokens) - context_length):
    y_offset = 2.2 - win_idx * 1.8

    # Input window (blue)
    for j in range(context_length):
        pos = win_idx + j
        rect = plt.Rectangle((pos * 1.5, y_offset), 1.2, 0.8,
                             facecolor='#3498db', alpha=0.3, edgecolor='#2980b9', linewidth=1.5)
        ax.add_patch(rect)
        ax.text(pos * 1.5 + 0.6, y_offset + 0.4, tokens[pos],
                ha='center', va='center', fontsize=10)

    # Target tokens (orange arrows)
    for j in range(context_length):
        pos = win_idx + j
        target_pos = pos + 1
        ax.annotate('', xy=(target_pos * 1.5 + 0.6, y_offset - 0.1),
                    xytext=(pos * 1.5 + 0.6, y_offset),
                    arrowprops=dict(arrowstyle='->', color='#e74c3c', lw=1.5))

    ax.text(-0.5, y_offset + 0.4, f'Window {win_idx+1}:', ha='right', va='center', fontsize=10)

ax.set_xlim(-2, len(tokens) * 1.5 + 0.5)
ax.set_ylim(-1, 5.5)
ax.set_aspect('equal')
ax.axis('off')
ax.set_title('Sliding Window: Input (blue) predicts next token (red arrows)', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.savefig('sliding_window.png', dpi=150, bbox_inches='tight')
plt.show()
print("Checkpoint: Each window creates a training sample where every position predicts the next token.")

In [ ]:
#@title 🎧 Listen: Math Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_07_math_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 3. The Mathematics

### Next-Token Prediction

For a sequence of tokens $x_1, x_2, \ldots, x_T$, the language model learns to maximize:

$$P(x_1, x_2, \ldots, x_T) = \prod_{t=1}^{T} P(x_t \mid x_1, \ldots, x_{t-1})$$

The training loss (cross-entropy) for a single sequence of length $T$ is:

$$\mathcal{L} = -\frac{1}{T} \sum_{t=1}^{T} \log P(x_t \mid x_1, \ldots, x_{t-1})$$

### Context Length and Memory

The context length $T$ determines how many tokens the model can see. The self-attention mechanism scales quadratically:

$$\text{Attention Memory} \propto T^2$$

Doubling $T$ from 1,024 to 2,048 means $4\times$ the attention memory.

### Batch Size and Gradient Variance

With batch size $B$, the variance of the gradient estimate decreases as:

$$\text{Var}[\hat{g}_B] = \frac{\text{Var}[\hat{g}_1]}{B}$$

So a batch of 32 has $32\times$ less gradient variance than a single sample.

In [ ]:
#@title 🎧 Code Walkthrough: Code Quadratic Scaling
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_08_code_quadratic_scaling.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Demonstrate the quadratic scaling of attention memory
context_lengths = [128, 256, 512, 1024, 2048, 4096, 8192]
relative_memory = [(T / 128) ** 2 for T in context_lengths]

print("Context Length vs Attention Memory:")
print("-" * 40)
for T, mem in zip(context_lengths, relative_memory):
    bar = "=" * int(min(mem, 50))
    print(f"  T = {T:>5,d}  ->  {mem:>7.0f}x  {bar}")

print("\nDoubling context length -> 4x memory!")

In [ ]:
#@title 🎧 Transition: Transition Build
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_09_transition_build.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 4. Let's Build It -- Component by Component

### 4.1 The TextDataset Class

In [ ]:
#@title 🎧 Code Walkthrough: Code Textdataset
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_10_code_textdataset.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    """
    A PyTorch Dataset for language model training.
    Creates input-target pairs using a sliding window over token sequences.
    """

    def __init__(self, tokens, context_length):
        """
        Args:
            tokens: List or tensor of token IDs
            context_length: Number of tokens in each training sample
        """
        self.tokens = torch.tensor(tokens, dtype=torch.long)
        self.context_length = context_length

    def __len__(self):
        """Number of training samples we can extract."""
        return len(self.tokens) - self.context_length

    def __getitem__(self, idx):
        """
        Get a single training sample.
        Input: tokens[idx : idx + context_length]
        Target: tokens[idx + 1 : idx + context_length + 1]
        """
        x = self.tokens[idx : idx + self.context_length]
        y = self.tokens[idx + 1 : idx + self.context_length + 1]
        return x, y

# Test with a small example
sample_tokens = list(range(20))  # [0, 1, 2, ..., 19]
dataset = TextDataset(sample_tokens, context_length=4)

print(f"Token sequence: {sample_tokens}")
print(f"Context length: 4")
print(f"Number of samples: {len(dataset)}")
print()

for i in range(min(5, len(dataset))):
    x, y = dataset[i]
    print(f"  Sample {i}: Input={x.tolist()}  Target={y.tolist()}")

In [ ]:
#@title 🎧 Listen: Explanation Batching
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_11_explanation_batching.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.2 Understanding Batching

Why can we not feed one sample at a time? Because single-sample gradients are extremely noisy. Batching averages the loss across multiple samples, producing smoother, more reliable gradients.

In [ ]:
#@title 🎧 What to Look For: Viz Batch Noise
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_12_viz_batch_noise.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Simulate gradient noise at different batch sizes
np.random.seed(42)
true_gradient = 0.5
num_experiments = 200

fig, axes = plt.subplots(1, 3, figsize=(15, 4))
batch_sizes = [1, 8, 32]

for ax, B in zip(axes, batch_sizes):
    # Simulate gradient estimates at this batch size
    gradient_estimates = [
        np.mean(np.random.normal(true_gradient, 2.0, size=B))
        for _ in range(num_experiments)
    ]

    ax.hist(gradient_estimates, bins=30, color='#3498db', edgecolor='black',
            linewidth=0.5, alpha=0.7, density=True)
    ax.axvline(true_gradient, color='red', linewidth=2, linestyle='--', label=f'True gradient = {true_gradient}')
    ax.set_title(f'Batch Size = {B}', fontsize=12, fontweight='bold')
    ax.set_xlabel('Gradient Estimate', fontsize=10)
    ax.set_ylabel('Density', fontsize=10)
    variance = np.var(gradient_estimates)
    ax.text(0.05, 0.95, f'Var = {variance:.3f}', transform=ax.transAxes,
            fontsize=10, verticalalignment='top', fontweight='bold')
    ax.legend(fontsize=9)
    ax.set_xlim(-4, 5)

plt.suptitle('Larger Batches Reduce Gradient Noise', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('batch_gradient_noise.png', dpi=150, bbox_inches='tight')
plt.show()
print("Checkpoint: Batch size 32 has ~32x less variance than batch size 1.")

In [ ]:
#@title 🎧 Listen: Explanation Dataloader
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_13_explanation_dataloader.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.3 The DataLoader

In [ ]:
#@title 🎧 Code Walkthrough: Code Dataloader
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_14_code_dataloader.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Create a DataLoader with batching and shuffling
dataset = TextDataset(list(range(100)), context_length=8)
dataloader = DataLoader(
    dataset,
    batch_size=4,       # 4 samples per batch
    shuffle=True,       # Randomize order each epoch
    drop_last=True,     # Drop incomplete final batch
)

print(f"Dataset size: {len(dataset)} samples")
print(f"Batch size: 4")
print(f"Number of batches: {len(dataloader)}")
print(f"Samples used per epoch: {len(dataloader) * 4}")
print(f"Dropped samples: {len(dataset) - len(dataloader) * 4}")

print("\nFirst 3 batches:")
for batch_idx, (batch_x, batch_y) in enumerate(dataloader):
    if batch_idx >= 3:
        break
    print(f"\n  Batch {batch_idx + 1}: shape = {batch_x.shape}")
    for i in range(len(batch_x)):
        print(f"    Sample {i}: Input={batch_x[i].tolist()[:4]}...  Target={batch_y[i].tolist()[:4]}...")

In [ ]:
#@title 🎧 Listen: Explanation Shuffling
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_15_explanation_shuffling.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.4 Why Shuffling Matters

In [ ]:
#@title 🎧 Code Walkthrough: Code Shuffling
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_16_code_shuffling.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Demonstrate the effect of shuffling
dataset = TextDataset(list(range(50)), context_length=4)

# Without shuffling
no_shuffle_loader = DataLoader(dataset, batch_size=8, shuffle=False)
print("WITHOUT shuffling (first batch each epoch):")
for epoch in range(3):
    first_batch = next(iter(no_shuffle_loader))
    print(f"  Epoch {epoch+1}: First input = {first_batch[0][0].tolist()}")

print()

# With shuffling
shuffle_loader = DataLoader(dataset, batch_size=8, shuffle=True)
print("WITH shuffling (first batch each epoch):")
for epoch in range(3):
    first_batch = next(iter(shuffle_loader))
    print(f"  Epoch {epoch+1}: First input = {first_batch[0][0].tolist()}")

print("\nShuffling ensures different batch composition each epoch!")

In [ ]:
#@title 🎧 Before You Start: Todo1 Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_17_todo1_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 5. Your Turn

### TODO 1: Implement a Streaming Dataset

Real LLM training data is too large to fit in memory. Implement a dataset that reads tokens from a file lazily.

In [ ]:
#@title 🎧 Before You Start: Todo1 Streaming Dataset
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_18_todo1_streaming_dataset.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
class StreamingTextDataset(Dataset):
    """
    A dataset that reads tokens from a memory-mapped file
    instead of loading everything into RAM.
    """

    def __init__(self, token_file_path, context_length):
        """
        Args:
            token_file_path: Path to a file containing token IDs (one per line or binary)
            context_length: Number of tokens per sample
        """
        self.context_length = context_length

        # TODO: Load token IDs -- for this exercise, simulate with a list
        # In production, you would use np.memmap or similar
        self.tokens = None  # YOUR CODE HERE: load tokens into a tensor

    def __len__(self):
        # TODO: Return the number of valid samples
        pass  # YOUR CODE HERE

    def __getitem__(self, idx):
        # TODO: Return (input, target) pair using sliding window
        pass  # YOUR CODE HERE

# SOLUTION (uncomment to verify):
# class StreamingTextDataset(Dataset):
#     def __init__(self, token_file_path, context_length):
#         self.context_length = context_length
#         # Simulate loading -- in production use np.memmap
#         self.tokens = torch.arange(10000, dtype=torch.long)
#     def __len__(self):
#         return len(self.tokens) - self.context_length
#     def __getitem__(self, idx):
#         x = self.tokens[idx : idx + self.context_length]
#         y = self.tokens[idx + 1 : idx + self.context_length + 1]
#         return x, y

In [ ]:
#@title 🎧 Before You Start: Todo2 Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_19_todo2_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### TODO 2: Implement Gradient Accumulation

When GPU memory is limited, we cannot use large batch sizes directly. Gradient accumulation simulates a larger batch by accumulating gradients over multiple forward-backward passes.

In [ ]:
#@title 🎧 Before You Start: Todo2 Gradient Accumulation
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_20_todo2_gradient_accumulation.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def train_with_gradient_accumulation(model, dataloader, optimizer, accumulation_steps=4):
    """
    Train with gradient accumulation.
    Effective batch size = dataloader.batch_size * accumulation_steps

    Args:
        model: The neural network
        dataloader: DataLoader with small batch size
        optimizer: The optimizer
        accumulation_steps: Number of mini-batches to accumulate before stepping
    """
    model.train()
    total_loss = 0

    for step, (batch_x, batch_y) in enumerate(dataloader):
        # TODO: Forward pass -- compute loss
        # loss = ...  # YOUR CODE HERE

        # TODO: Scale loss by accumulation_steps (so gradients average correctly)
        # scaled_loss = ...  # YOUR CODE HERE
        # scaled_loss.backward()

        # TODO: Only step the optimizer every accumulation_steps mini-batches
        # if (step + 1) % accumulation_steps == 0:
        #     optimizer.step()
        #     optimizer.zero_grad()

        # total_loss += loss.item()
        pass

    # SOLUTION (uncomment to verify):
    # for step, (batch_x, batch_y) in enumerate(dataloader):
    #     logits = model(batch_x)
    #     loss = F.cross_entropy(logits.view(-1, logits.size(-1)), batch_y.view(-1))
    #     scaled_loss = loss / accumulation_steps
    #     scaled_loss.backward()
    #     if (step + 1) % accumulation_steps == 0:
    #         optimizer.step()
    #         optimizer.zero_grad()
    #     total_loss += loss.item()

    print("Gradient accumulation simulates larger batch sizes without more memory!")

In [ ]:
#@title 🎧 Transition: Transition Full Pipeline
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_21_transition_full_pipeline.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 6. Putting It All Together

Let us build a complete data pipeline from raw text to training-ready batches.

In [ ]:
#@title 🎧 Code Walkthrough: Code Full Pipeline
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_22_code_full_pipeline.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
import subprocess
subprocess.check_call(['pip', 'install', '-q', 'tiktoken'])
import tiktoken

# Step 1: Raw text
raw_text = """
The transformer architecture has revolutionized natural language processing.
Self-attention allows the model to weigh the importance of different tokens.
Training a language model requires tokenization, batching, and optimization.
The data loader feeds organized batches to the GPU for efficient training.
Each training step computes loss, backpropagates, and updates weights.
""" * 50  # Repeat to get more data

# Step 2: Tokenize
enc = tiktoken.get_encoding("gpt2")
tokens = enc.encode(raw_text)
print(f"Raw text: {len(raw_text)} characters")
print(f"Tokens: {len(tokens)} token IDs")
print(f"Compression ratio: {len(raw_text) / len(tokens):.1f} chars/token")

# Step 3: Create dataset
CONTEXT_LENGTH = 32
dataset = TextDataset(tokens, context_length=CONTEXT_LENGTH)
print(f"\nDataset: {len(dataset)} training samples")
print(f"Context length: {CONTEXT_LENGTH}")

# Step 4: Create dataloader
BATCH_SIZE = 8
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
print(f"Batches per epoch: {len(dataloader)}")
print(f"Effective training tokens per epoch: {len(dataloader) * BATCH_SIZE * CONTEXT_LENGTH:,}")

# Step 5: Verify shapes
batch_x, batch_y = next(iter(dataloader))
print(f"\nBatch shapes:")
print(f"  Input:  {batch_x.shape}  (batch_size x context_length)")
print(f"  Target: {batch_y.shape}  (batch_size x context_length)")

# Decode one sample to verify correctness
sample_input = enc.decode(batch_x[0].tolist())
sample_target = enc.decode(batch_y[0].tolist())
print(f"\nSample input:  '{sample_input[:60]}...'")
print(f"Sample target: '{sample_target[:60]}...'")
print("\nTarget is input shifted right by 1 token -- correct!")

In [ ]:
#@title 🎧 Transition: Transition Benchmark
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_23_transition_benchmark.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 7. Training and Results

Let us measure the throughput of our data pipeline -- how fast can it feed data?

In [ ]:
#@title 🎧 Code Walkthrough: Code Benchmark
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_24_code_benchmark.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
import time

# Benchmark data loading speed
dataset_large = TextDataset(list(range(100000)), context_length=128)

for batch_size in [8, 32, 128]:
    loader = DataLoader(dataset_large, batch_size=batch_size, shuffle=True,
                        drop_last=True, num_workers=0)

    start = time.time()
    tokens_loaded = 0
    for batch_x, batch_y in loader:
        tokens_loaded += batch_x.numel()
    elapsed = time.time() - start

    print(f"Batch size {batch_size:>4d}:  {tokens_loaded:>10,d} tokens in {elapsed:.3f}s  "
          f"({tokens_loaded / elapsed / 1e6:.1f}M tokens/sec)")

print("\nThe data pipeline must be fast enough that the GPU never waits for data!")

In [ ]:
#@title 🎧 What to Look For: Final Output Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_25_final_output_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 8. Final Output

In [ ]:
#@title 🎧 What to Look For: Viz Pipeline Summary
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_26_viz_pipeline_summary.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Summary visualization of the complete data pipeline
fig, axes = plt.subplots(1, 4, figsize=(16, 3))

stages = [
    ('Raw Text', 'characters\n"The cat sat..."', '#e8f4f8'),
    ('Tokenizer', 'token IDs\n[464, 3797, 3332]', '#fde8e8'),
    ('Dataset', 'sliding window\nInput -> Target', '#e8fde8'),
    ('DataLoader', 'shuffled batches\nShape: (B, T)', '#fdf8e8'),
]

for ax, (title, desc, color) in zip(axes, stages):
    rect = plt.Rectangle((0.05, 0.1), 0.9, 0.8, facecolor=color,
                         edgecolor='black', linewidth=2, transform=ax.transAxes)
    ax.add_patch(rect)
    ax.text(0.5, 0.7, title, ha='center', va='center', fontsize=13,
            fontweight='bold', transform=ax.transAxes)
    ax.text(0.5, 0.35, desc, ha='center', va='center', fontsize=9,
            transform=ax.transAxes, style='italic')
    ax.axis('off')

# Add arrows between stages
for i in range(3):
    fig.text(0.25 * (i + 1) + 0.02, 0.5, '>>>', ha='center', va='center',
             fontsize=16, fontweight='bold', color='#e74c3c')

plt.suptitle('The Data Pipeline: Raw Text to Training Batches', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('data_pipeline.png', dpi=150, bbox_inches='tight')
plt.show()
print("Data pipeline complete! Ready to feed the optimizer.")

In [ ]:
#@title 🎧 Wrap-Up: Closing
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_27_closing.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 9. Reflection and Next Steps

**What we built:** A complete data pipeline -- from raw text through tokenization, dataset creation, and batch loading.

**Key takeaways:**
- The Dataset implements a sliding window for next-token prediction
- Target = Input shifted right by 1 token
- Larger batches reduce gradient variance (smoother training)
- Shuffling prevents order-dependent memorization
- Context length trades off memory (quadratic in attention) for long-range understanding

**What is next:** In Notebook 3, we will build the optimizers -- SGD, Momentum, and Adam -- that use these batches to update model weights.